In [1]:
%pip install -U sagemaker
%pip install -U xgboost
%pip install -U jinja2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 137.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 137.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.112
    Uninstalling botocore-1.34.112:
      Successfully uninstalled botocore-1.34.112
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.112
    Uninstalling boto3-1.34.112:
      Successfully uninstalled boto3-1.34.112
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.221.1
    Uninstalling sagemaker-2.221.1:
      Successfully uninstalled sagemaker-2.221.1
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.

In [22]:
import sagemaker
from sagemaker.multidatamodel import MultiDataModel
from sagemaker.model import Model

from pathlib import Path
import boto3
import json
import shutil
import datetime as dt
import tarfile
import xgboost as xgb
import pandas as pd
import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor


In [4]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

triton_framework = "sagemaker-tritonserver"
version = "24.09"
instance_type = "ml.g5.2xlarge"

test_data_path = (
    "s3://sagemaker-us-east-1-152804913371/nvidia-aws-fraud-detection-demo/test/"
)
trained_model_path = "s3://sagemaker-us-east-1-152804913371/pytorch-training-2024-11-22-15-54-43-056/output/model.tar.gz"

mme_s3_uri = f"s3://{sess.default_bucket()}/xgboost-mme"

mme_triton_image_uri = sagemaker.image_uris.retrieve(
    framework=triton_framework,
    region=region,
    version=version,
    instance_type=instance_type,
)

In [5]:
model_tar_name = Path(trained_model_path).name
!aws s3 cp {trained_model_path} {model_tar_name}
with tarfile.open(model_tar_name, "r:gz") as tar:
    model_file_name = tar.getnames()[0]
    tar.extractall()

print(f"Model file name: {model_file_name}")

download: s3://sagemaker-us-east-1-152804913371/pytorch-training-2024-11-22-15-54-43-056/output/model.tar.gz to ./model.tar.gz
Model file name: xgboost.json


In [6]:
bst = xgb.Booster()
bst.load_model(model_file_name)
num_features = bst.num_features()

In [7]:
from jinja2 import Template

config_template = """backend: "fil"
max_batch_size: 1000
input [                                 
 {  
  name: "input__0"
  data_type: TYPE_FP32
  dims: [ {{ input_size }} ]                    
  } 
]
output [
 {
  name: "output__0"
  data_type: TYPE_FP32
  dims: [ 2 ]
  }
]
instance_group [{ kind: KIND_{{ device }} }]
parameters [
  {
  key: "model_type"
  value: { string_value: "xgboost_json" }
  },
  {
  key: "predict_proba"
  value: { string_value: "true" }
  },
  {
  key: "output_class"
  value: { string_value: "true" }
  },
  {
  key: "threshold"
  value: { string_value: "0.5" }
  },
  {
  key: "storage_type"
  value: { string_value: "AUTO" }
  }
]

dynamic_batching {

}
"""

template = Template(config_template)

In [8]:
model_workspace = Path("workspace")
gpu_model_path = model_workspace / "xgboost_gpu"
cpu_model_path = model_workspace / "xgboost_cpu"

for device, model_path in zip(["GPU", "CPU"], [gpu_model_path, cpu_model_path]):
    model_path.mkdir(parents=True, exist_ok=True)
    with open(model_path / "config.pbtxt", "w") as f:
        f.write(
            template.render(
                input_size=num_features,
                device=device,
            )
        )
    (model_path / "1").mkdir(parents=True, exist_ok=True)
    shutil.copy(model_file_name, model_path / "1" / model_file_name) 

cpu_tar_name = "xgboost_cpu.tar.gz"
gpu_tar_name = "xgboost_gpu.tar.gz"

with tarfile.open(cpu_tar_name, "w:gz") as tar:
    tar.add(cpu_model_path, arcname=cpu_model_path.name)

with tarfile.open(gpu_tar_name, "w:gz") as tar:
    tar.add(gpu_model_path, arcname=gpu_model_path.name)


In [9]:
gpu_s3_uri = sess.upload_data(gpu_tar_name, bucket=sess.default_bucket(), key_prefix="xgboost-mme")
cpu_s3_uri = sess.upload_data(cpu_tar_name, bucket=sess.default_bucket(), key_prefix="xgboost-mme")

In [10]:
model = Model(
    model_data=cpu_s3_uri,
    image_uri=mme_triton_image_uri,
    role=role,
    sagemaker_session=sess
)

mme = MultiDataModel(
    name="triton-fil-" + dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"),
    model_data_prefix=mme_s3_uri,
    model=model,
    sagemaker_session=sess,
)

In [11]:
mme.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
)

[11/26/24 20:25:27] INFO     Creating model with name: triton-fil-2024-11-26-20-25-24               ]8;id=545124;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=861209;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4025\4025]8;;\

                    INFO     Creating endpoint-config with name triton-fil-2024-11-26-20-25-24      ]8;id=519670;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=799083;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#5820\5820]8;;\

[11/26/24 20:25:28] INFO     Creating endpoint with name triton-fil-2024-11-26-20-25-24             ]8;id=752539;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=201503;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4642\4642]8;;\

-----------!

In [13]:
list(mme.list_models())

['/xgboost_cpu.tar.gz', '/xgboost_gpu.tar.gz']

In [14]:
predictor = sagemaker.predictor.Predictor(endpoint_name=mme.endpoint_name, sagemaker_session=sess)

In [15]:
test_data_bucket = test_data_path.split("/")[2]
test_data_prefix = "/".join(test_data_path.split("/")[3:])


test_file = sess.list_s3_files(test_data_bucket, test_data_prefix)[0]
test_file =f"s3://{test_data_bucket}/{test_file}"

In [16]:
df = pd.read_parquet(test_file)

In [17]:
df.drop(columns=["TX_FRAUD_1"], inplace=True)

In [18]:
def prepare_payload(input_data: np.ndarray, num_features: int) -> dict:

    input_data = input_data.reshape(-1, num_features)
    
    payload = {
        "inputs": [
            {
                "name": "input__0",
                "shape": input_data.shape,
                "datatype": "FP32",
                "data": input_data.tolist(),
            }
        ]
    }
    return json.dumps(payload)

def make_batches (data: np.ndarray, batch_size: int) -> list:
    return [data[i:i+batch_size] for i in range(0, len(data), batch_size)]

In [26]:
BATCH_SIZE = 500
payloads = [prepare_payload(batch, num_features=32) for batch in make_batches(df.values, BATCH_SIZE)]

In [27]:
NUM_CLIENTS = 10
from functools import partial

start_time = time.perf_counter()
with ThreadPoolExecutor(max_workers=NUM_CLIENTS) as executor:
    cpu_predict = partial(predictor.predict, target_model="/xgboost_cpu.tar.gz")
    results = list(executor.map(cpu_predict, payloads))
cpu_throughput = len(df) / (time.perf_counter() - start_time)
print(f"CPU throughput: {cpu_throughput:.2f} records per second")

start_time = time.perf_counter()
with ThreadPoolExecutor(max_workers=NUM_CLIENTS) as executor:
    cpu_predict = partial(predictor.predict, target_model="/xgboost_gpu.tar.gz")
    results = list(executor.map(cpu_predict, payloads))
cpu_throughput = len(df) / (time.perf_counter() - start_time)
print(f"GPU throughput: {cpu_throughput:.2f} records per second")

CPU throughput: 123673.93 records per second
GPU throughput: 348148.16 records per second


In [ ]:
mme.delete_endpoint()